In [1]:
import pinecone
from llama_index import StorageContext
from llama_index.vector_stores import PineconeVectorStore

# pinecone_api_key = os.environ.get("PINECONE_API_KEY")
# pinecone_env = os.environ.get("PINECONE_ENV")

# pinecone_api_key = os.environ['PINECONE_API_KEY']
# pinecone_env = os.environ['PINECONE_ENV']
pinecone_api_key = "29446de0-e17c-419e-b2f6-7d3b9e30a9a8"
pinecone_env = "asia-southeast1-gcp"
# init pinecone
pinecone.init(api_key=pinecone_api_key, environment=pinecone_env)

d:\anaconda3\envs\langchain\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
langchain_module_path = os.path.abspath(os.path.join('../LangChainStudy/'))
model_config_path = os.path.abspath(os.path.join('../LangChainStudy/custom_llms/'))
sys.path.insert(1, module_path)
sys.path.insert(1, langchain_module_path)
sys.path.insert(1, model_config_path)

from custom_llms import (
    ZhipuAIEmbeddings,
    Zhipuai_LLM,
    load_api
)
api_key = load_api()
model = Zhipuai_LLM(zhipuai_api_key=api_key)

In [12]:
pinecone.create_index("xdtest1", dimension=1024, metric="cosine", pod_type="p1") # pinecone.create_index("quickstart", dimension=1536, metric="euclidean", pod_type="p1")

In [13]:
# construct vector store and customize storage context
storage_context = StorageContext.from_defaults(
    vector_store = PineconeVectorStore(pinecone.Index("xdtest1"))
)

In [14]:
# Load documents and build index
from llama_index import SimpleDirectoryReader, ServiceContext, VectorStoreIndex
from llama_index import GPTVectorStoreIndex
documents_p = SimpleDirectoryReader('../data/mr_fujino').load_data()


In [15]:
service_context = ServiceContext.from_defaults(llm=model, embed_model=ZhipuAIEmbeddings())

In [16]:
index_p = VectorStoreIndex.from_documents(documents_p, service_context=service_context, storage_context=storage_context)

Upserted vectors: 100%|██████████| 5/5 [00:01<00:00,  2.73it/s]


In [17]:
query_engine = index_p.as_query_engine()

In [18]:
response = query_engine.query("下面鲁迅先生以第一人称‘我’写的内容，请你用中文总结一下:")

print(response)

这段文字是鲁迅以第一人称“我”叙述自己在日本留学期间的经历。主要内容包括：\n\n1. 鲁迅在仙台医学专门学校学习，成绩优异，但引起了一些嫉妒和误解。有人认为他的高分是得益于藤野先生泄露试题，这是一种侮辱。\n\n2. 鲁迅将此事告诉了藤野先生，并得到一些同学的支持。他们一起去责问干事，要求公布检查结果。最后，流言得以消除，但干事却试图收回匿名信。鲁迅将信退还给他们。\n\n3. 在学校里，鲁迅受到了歧视。有一次看电影，其中涉及到中国人为敌方做侦探被俘虏枪毙的情节，学生们欢呼庆祝。这使鲁迅深感刺耳，对国家民族的命运感到悲痛。\n\n4. 鲁迅在藤野先生的指导下学习解剖学等课程。尽管成绩优秀，但他内心并不热爱医学。在离开仙台前，鲁迅告诉藤野先生自己想转学生物学，并归还了一封写着“惜别”的照相。\n\n5. 鲁迅离开仙台后，回顾自己在那里的经历，对藤野先生的感激和不舍之情油然而生。尽管在异国他乡遭受歧视和误解，但这些经历使他更加坚定地关注国家民族的命运。
